In [2]:
from __future__ import division
import numpy as np
import pandas as pd
import holoviews as hv
from IPython.display import display_html
import holoviews.plotting.mpl
list(hv.Store.registry['matplotlib'].items())[0:5]
from bokeh.embed import *
from bokeh.io import *
from bokeh.models import *
from bokeh.plotting import *
from bokeh.palettes import *
from bokeh.transform import *
import holoviews.plotting.mpl
from bokeh.models.widgets import *
from bokeh.layouts import *
import math
import io

In [3]:
#Importing dataframes 
countries = pd.read_csv('/Users/noraamri/Documents/mastars  2nd semester /visualisation /Project /data/countries of the world 3.csv')
countries = countries[['Country','ID','Population','GDP ($ per capita)','Area (sq. mi.)']].drop_duplicates()
countries['Country']= countries['Country'].str.strip()
airports_num = pd.read_csv('/Users/noraamri/Documents/mastars  2nd semester /visualisation /Project /data/airports_num.csv')

#Merging the two dataframes 
countries_total = pd.merge(countries,airports_num[['Airports number']],
                            right_on = airports_num['Country'],
                            left_on=['Country'],
                            how='left')
countries_total = countries_total[['Country','ID','Population','GDP ($ per capita)','Area (sq. mi.)', 'Airports number']].drop_duplicates().dropna()
countries_continents = pd.read_csv('/Users/noraamri/Documents/mastars  2nd semester /visualisation /Project /data/countries_continents.csv')

#saving the new dataframe
countries_total.to_csv('/Users/noraamri/Documents/mastars  2nd semester /visualisation /Project /data/countries_total1.csv')

#creating a dictonery in order to use it later in graphs 
source1 = ColumnDataSource(data=dict(
    Country=countries_continents['Country'],
    ID=countries_continents['ID'],
    Population=countries_continents['Population'],
    GDP =countries_continents['GDP ($ per capita)'],
    Area =countries_continents['Area (sq. mi.)'],
    Airports_num = countries_continents['Airports number'],
    sizes2=[ math.sqrt(x)/50 for x in countries_continents['Area (sq. mi.)']],
    Continent= countries_continents['Continent_Name'] ))


Airports_num_av = countries_total['Airports number'].mean()
GDP_av=countries_total['GDP ($ per capita)'].mean()
Area_av= [math.sqrt(countries_total['Area (sq. mi.)'].mean())/50]
pop_av= countries_total['Population'].mean()


countries_total['Airports number'].max()
countries_total['GDP ($ per capita)'].max()
countries_total.head()

countries_total['Population'].min()


7502

In [4]:
#Creating lists in order to use them later in graphs 

area = countries_continents['Area (sq. mi.)'].dropna()
areasizelist = area.values.tolist()
sizes2 = [ math.sqrt(x)/50 for x in areasizelist]  

In [5]:
#creating the Bubble chart 

tooltips = [
    ('Country', '@Country'),
    ('Airports number', '@Airports_num'),
    ('GDP', '@GDP ($ per capita)'),
    ('Population', '@Population'),
    ('Area (sq. mi.)', '@Area (sq. mi.)'),
    ('Continent', '@Continent')]

hv = figure(plot_width=1200,
            plot_height=620,
            title='World airports in comparison',
            tooltips = tooltips,
            toolbar_location="above",
            y_range=(0, 600))

mapper = log_cmap(field_name='Population',
                  palette=Spectral6 ,
                  low=min(countries_continents['Population']),
                  high=max(countries_continents['Population']))

color_bar = ColorBar(color_mapper=mapper['transform'],
                     width=8,
                     location=(0,0),
                     ticker=LogTicker(),
                     label_standoff=12)

points = hv.circle('GDP','Airports_num',
                   source = source1,
                   size= 'sizes2',
                   line_color=mapper,
                   color=mapper,
                   fill_alpha= 0.5)

#setting the layout properties 

hv.xaxis[0].formatter = NumeralTickFormatter(format="$0")
hv.add_layout(color_bar, 'right')
hv.xaxis.axis_label = "GDP ($ per capita)"
hv.yaxis.axis_label = "Number of Airports"
hv.add_layout(Title(text="Population", align="center"), "right")
hover = HoverTool(tooltips=tooltips)
hv.y_range.end = 550

#creating the filtering table
columns = [
    TableColumn(field="Country", title="Country"),
    TableColumn(field="Population", title="Population"),
    TableColumn(field="Area", title="Area (sq. mi.)"),
    TableColumn(field="GDP", title="@GDP ($ per capita)"),
    TableColumn(field="Airports_num",title="Airports number"),
    TableColumn(field="Continent", title="Continent")]

data_table = DataTable(source=source1,
                       columns=columns,
                       width=1200,
                       height=280)


show(column(hv,data_table))